In [9]:
import numpy as np
import pandas as pd

In [10]:
train = pd.read_csv('https://media-doselect.s3.amazonaws.com/generic/rONqp9gR3oaWE4pWP3WKedr1Z/train.csv')
test = pd.read_csv('https://media-doselect.s3.amazonaws.com/generic/dO7Ag1214breGowW1JAOQVWZ/test.csv',header=None)

In [11]:
train.head()

,Stock_price
0,1455.219971
1,1399.420044
2,1402.109985
3,1403.449951
4,1441.469971


In [12]:
test.head()

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
0,0.537575,0.546623,0.558551,0.576343,0.599334,0.607099,0.593685,0.591434,0.551676,0.509813,...,0.606694,0.581182,0.575623,0.546083,0.542572,0.580698,0.583669,0.641928,0.639250,0.638912
1,0.546623,0.558551,0.576343,0.599334,0.607099,0.593685,0.591434,0.551676,0.509813,0.517544,...,0.581182,0.575623,0.546083,0.542572,0.580698,0.583669,0.641928,0.639250,0.638912,0.653316
2,0.558551,0.576343,0.599334,0.607099,0.593685,0.591434,0.551676,0.509813,0.517544,0.547388,...,0.575623,0.546083,0.542572,0.580698,0.583669,0.641928,0.639250,0.638912,0.653316,0.654881
3,0.576343,0.599334,0.607099,0.593685,0.591434,0.551676,0.509813,0.517544,0.547388,0.561376,...,0.546083,0.542572,0.580698,0.583669,0.641928,0.639250,0.638912,0.653316,0.654881,0.657739
4,0.599334,0.607099,0.593685,0.591434,0.551676,0.509813,0.517544,0.547388,0.561376,0.534008,...,0.542572,0.580698,0.583669,0.641928,0.639250,0.638912,0.653316,0.654881,0.657739,0.627737


In [13]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0,1))
train_scaled = sc.fit_transform(train)

In [14]:
X_train = []
y_train = []
input_features = len(train)

for i in range(60, input_features): 
    X_train.append(train_scaled[i-60:i, 0]) 
    y_train.append(train_scaled[i, 0]) 
X_train, y_train = np.array(X_train), np.array(y_train) 
#X_train.shape
# Reshaping
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [15]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [16]:
# Initialising the RNN
model = Sequential()

# Adding the first LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
model.add(Dropout(0.2))

# Adding a second LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))

# Adding a third LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))

# Adding a fourth LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50))
model.add(Dropout(0.2))

# Adding the output layer
model.add(Dense(units = 1))

# Compiling the RNN
model.compile(loss = 'mean_squared_error', optimizer='sgd', metrics=['mae', 'acc'])

# Fitting the RNN to the Training set
model.fit(X_train, y_train, epochs = 12, batch_size = 32)

Epoch 1/12
2940/2940 [==============================] - 41s 14ms/step - loss: 0.0688 - mean_absolute_error: 0.2050 - acc: 0.0000e+00
Epoch 2/12
2940/2940 [==============================] - 39s 13ms/step - loss: 0.0410 - mean_absolute_error: 0.1630 - acc: 3.4014e-04
Epoch 3/12
2940/2940 [==============================] - 39s 13ms/step - loss: 0.0384 - mean_absolute_error: 0.1581 - acc: 3.4014e-04
Epoch 4/12
2940/2940 [==============================] - 39s 13ms/step - loss: 0.0360 - mean_absolute_error: 0.1532 - acc: 3.4014e-04
Epoch 5/12
2940/2940 [==============================] - 39s 13ms/step - loss: 0.0320 - mean_absolute_error: 0.1435 - acc: 6.8027e-04
Epoch 6/12
2940/2940 [==============================] - 39s 13ms/step - loss: 0.0278 - mean_absolute_error: 0.1336 - acc: 6.8027e-04
Epoch 7/12
2940/2940 [==============================] - 39s 13ms/step - loss: 0.0219 - mean_absolute_error: 0.1184 - acc: 6.8027e-04
Epoch 8/12
2940/2940 [==============================] - 39s 13ms/step

In [17]:
X_test = np.array(test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [18]:
y_pred = model.predict(X_test)

In [19]:
#y_pred

In [20]:
pred_df = pd.DataFrame(y_pred, columns=['predictions'])

In [21]:
pred_df.head()

,predictions
0,0.620271
1,0.618482
2,0.617003
3,0.615926
4,0.615257


In [22]:
pred_df.to_csv('/home/user/workspace/predictions.csv')